In [1]:
import pandas as pd

In [2]:
df_de = pd.read_csv('../grams/sg_de.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams
df_nl = pd.read_csv('../grams/sg_nl.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams

In [3]:
# list of ~94k US babynames to remove (hello, john), (james, bond)
names = pd.read_csv("../names.csv", names=['name'])['name'].tolist()

In [4]:
names = [i for i in names if type(i) == str] #remove non strings

# Filter names and from list

In [5]:
german_filter = ['ich', 'du', 'sie', 'mir', 'wir', 'ihr', 'uns', 'euch', 'die', 'das', 'der', 'den', 'er', 'und', '!', '/', 'ein', 'eine', 'einer', 'einem', 'mein', 'dein', 'nicht', 'wie', 'wo', 'wann', 'was']

In [6]:
dutch_filter = ['ik', 'jij', 'hij', 'zij', 'wij', 'we', 'jullie', 'hun', 'mij', 'me', 'mijn', 'haar', 'hem', '!', '/', 'een', 'de', 'het', 'wie', 'wat', 'waar', 'hoe', 'niet']

In [7]:
french_filter = ['un', 'une', 'le', 'la', '!', '/', 'je', 'tu', 'nous', 'vous']

In [8]:
spanish_filter = ['!', '/', 'el']

In [9]:
l_de = names + german_filter
l_nl = names + dutch_filter

In [10]:
df_de = \
    df_de[ \
    ~df_de.word1.isin(l_de) & \
    ~df_de.word2.isin(l_de) & \
    ~df_de.word1.str.isnumeric() & \
    ~df_de.word2.str.isnumeric() &\
    ~(df_de.word1.str == df_de.word2.str)
    ]
    
df_nl = \
    df_nl[ \
    ~df_nl.word1.isin(l_nl) & \
    ~df_nl.word2.isin(l_nl) & \
    ~df_nl.word1.str.isnumeric() & \
    ~df_nl.word2.str.isnumeric() & \
    ~(df_de.word1.str == df_de.word2.str)
    ]

# Remove words overlapping with other languages

In [11]:
df_de_ex = df_de \
    .merge(df_nl ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left') \
    .merge(df_fr ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left') \
    .merge(df_es ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left')

In [12]:
df_nl_ex = df_nl \
    .merge(df_de ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left')

In [13]:
df_nl_ex.frequency_y.fillna(0, inplace=True)

In [14]:
df = df_nl_ex[df_nl_ex.frequency_y == 0.0].drop('frequency_y', axis=1).rename(index=str, columns={"frequency_x": "frequency"})

# Add word frequency

In [15]:
wc_de = pd.read_csv("../word_counts/wc_de.csv", lineterminator='\n', names=['word', 'word_frequency'])

In [16]:
df = df.merge(wc_de, left_on='word1', right_on='word', how='left') \
.merge(wc_de, left_on='word2', right_on='word', how='left') \
.drop(['word_x', 'word_y'], axis=1) \
.rename(index=str, columns={"word_frequency_x": "word_1_frequency", "word_frequency_y": "word_2_frequency"})

In [17]:
df['normalised_frequency'] = df['frequency'] / (df['word_1_frequency']+df['word_2_frequency']) # divide skipgram freq. by wordcount freq.

In [19]:
df.sort_values(by='frequency', ascending=False)

,frequency,word1,word2,word_1_frequency,word_2_frequency,normalised_frequency
11258,607621,es,ist,3196654,4236656,0.081743
83420,306065,es,zu,3196654,1991560,0.058992
76626,260714,ist,es,4236656,3196654,0.035074
149440,256849,um,zu,583880,1991560,0.099730
168544,209711,ist,zu,4236656,1991560,0.033671
89109,182353,tut,leid,240465,194850,0.418899
1449,161627,wenn,es,971053,3196654,0.038781
67009,161274,es,war,3196654,988962,0.038531
79341,145130,es,gibt,3196654,277759,0.041771
4213,142900,dass,es,1195857,3196654,0.032533
